VMD分解:使所有信号的分量特征保持同样的维度，K= 4

快速傅里叶变换 FFT

In [1]:
# VMD + FFT 数据集的制作
import pandas as pd
import numpy as np
from vmdpy import VMD
import torch
from joblib import dump, load
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# 加载数据
train_xdata = load('train_xdata')
val_xdata = load('val_xdata')
test_xdata = load('test_xdata')
train_ylabel = load('train_ylabel')
val_ylabel = load('val_ylabel')
test_ylabel = load('test_ylabel')

# -----VMD 参数--------
alpha = 2000
tau = 0 
DC = 0 
init = 1 
tol = 1e-7
# -----参数--------

# VMD + FFT 数据集的制作
def make_feature_datasets(data, imfs_k):
    '''
        参数 data: 待分解数据
            imfs_k: VMD分解模态（IMF）个数

        返回 features:  提取的特征数据集
            y_label    : 数据集对应标签
    '''
    samples = data.shape[0]
    signl_len = data.shape[1]
    # 把数据转为numpy
    data = np.array(data)

    # 特征数量 分解数量  + 快速傅里叶变换结果
    features_num = imfs_k + 1

    # 构造三维矩阵
    features = np.zeros((samples, features_num, signl_len))  

    # 对数据进行VMD分解 和 FFT 变换
    for i in range(samples):

        # Apply VMD
        u, u_hat, omega= VMD(data[i], alpha, tau, imfs_k, DC, init, tol)

        # 快速傅里叶变换
        fft_result1 = np.fft.fft(data[i])
        magnitude_spectrum = np.abs(fft_result1)
        magnitude_spectrum = magnitude_spectrum.reshape(1, -1)

        # 垂直叠加
        combined_matrix = np.vstack((u, magnitude_spectrum))
        features[i] = combined_matrix

    # 把numpy转为  tensor
    features = torch.tensor(features).float()
    return features

# VMD分解预处理  统一保存4个分量
#模态数量  分解模态（IMF）个数
K = 4

train_features = make_feature_datasets(train_xdata, K)
val_features = make_feature_datasets(val_xdata, K)
test_features = make_feature_datasets(test_xdata, K)

# 保存数据
dump(train_features, 'train_features_vmd_fft')
dump(val_features, 'val_features_vmd_fft')
dump(test_features, 'test_features_vmd_fft')

['test_features_vmd_fft']

In [2]:
print('数据 形状：')
print(train_features.shape, train_ylabel.shape)
print(val_features.shape, val_ylabel.shape)
print(test_features.shape, test_ylabel.shape)

数据 形状：
torch.Size([7161, 5, 1024]) torch.Size([7161])
torch.Size([2046, 5, 1024]) torch.Size([2046])
torch.Size([1023, 5, 1024]) torch.Size([1023])
